In [2]:
import pickle
import pandas as pd
import csv

results = list()

datasetFile = open("fnmr/angles/dataset/fnmr_dataset_multipie_angles.pickle", 'rb')
dataset = pickle.load(datasetFile)

resultFile = open("fnmr/angles/results/openface_fnmr_multipie_angles.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/angles/results/arcface_fnmr_multipie_angles.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/angles/results/facenet_fnmr_multipie_angles.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/angles/results/vggface_fnmr_multipie_angles.pickle", 'rb')
results.append(pickle.load(resultFile))

resultFile = open("fnmr/angles/results/faceapi_fnmr_multipie_angles.pickle", 'rb')
faceapiDataset = (pickle.load(resultFile))


resultDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for result in results:
    for algorithm in result.keys():
        if result[algorithm]:
            resultDict[algorithm] = result[algorithm]

In [3]:
processDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict()
}

for algorithm in resultDict.keys():
    for index in range(len(dataset)):
        referenceSubject = (dataset[index][0].split('/'))[3]
        angle = (dataset[index][1].split('/'))[5]

        pairNumber = 'pair_' + str(index + 1)
        if not(referenceSubject in processDict[algorithm].keys()):
            processDict[algorithm][referenceSubject] = dict()
        if not(angle in processDict[algorithm][referenceSubject].keys()):
            processDict[algorithm][referenceSubject][angle] = list()
        processDict[algorithm][referenceSubject][angle].append(resultDict[algorithm][pairNumber])

faceAPIResult = dict()

for comparison in faceapiDataset:
    parsedComparison = comparison.split(" ")
    parsedComparison[0] = parsedComparison[0].replace("/home/ubuntu/Multipie/Multi-Pie/", "")
    parsedComparison[1] = parsedComparison[1].replace("/home/ubuntu/Multipie/Multi-Pie/", "")
    parsedComparison[2].replace("undefined", "1")
    try: 
        parsedComparison[2] = float(parsedComparison[2])
    except:
        parsedComparison[2] = float(1)

    
    referenceSubject = (parsedComparison[0].split('/'))[3]
    angle = (parsedComparison[1].split('/'))[5]
    distance = parsedComparison[2]

    comparisonDict = dict()
    comparisonDict["distance"] = distance
    if distance >= float(.6):
        comparisonDict["verified"] = False
    else:
        comparisonDict["verified"] = True

    if not(referenceSubject in faceAPIResult.keys()):
        faceAPIResult[referenceSubject] = dict()
    if not(angle in faceAPIResult[referenceSubject].keys()):
        faceAPIResult[referenceSubject][angle] = list()

    faceAPIResult[referenceSubject][angle].append(comparisonDict)

processDict["FaceAPI"] = faceAPIResult


In [4]:
verifiedDict = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

angleNames = {
    "11_0": "-90",
    "12_0": "-75",
    "09_0": "-60",
    "08_0": "-45",
    "13_0": "-30",
    "14_0": "-15",
    "05_1": "0",
    "05_0": "15",
    "04_1": "30",
    "19_0": "45",
    "20_0": "60",
    "01_0": "75",
    "24_0": "90"
}

for algorithm in processDict.keys():
    for subject in processDict[algorithm].keys():
        if not(subject in verifiedDict[algorithm].keys()):
            verifiedDict[algorithm][subject] = dict()
        for angle in processDict[algorithm][subject].keys():
            if not(angleNames[angle] in verifiedDict[algorithm][subject].keys()):
                verifiedDict[algorithm][subject][angleNames[angle]] = list()
            for comparison in processDict[algorithm][subject][angle]:
                verified = False
                if comparison["distance"] < .6:
                    verified = True
                verifiedDict[algorithm][subject][angleNames[angle]].append(verified)



In [5]:
verifiedStats = {
    "Facenet": dict(),
    "VGG-Face": dict(),
    "OpenFace": dict(),
    "ArcFace": dict(),
    "FaceAPI": dict()
}

for algorithm in verifiedDict.keys():
    verifiedStats[algorithm]["tpr"] = 0
    verifiedStats[algorithm]["fnr"] = 0
    verifiedStats[algorithm]["angle"] = dict()
    for angle in angleNames.keys():
        if angle == "05_1":
            continue
        verifiedStats[algorithm]["angle"][angleNames[angle]] = dict()
        verifiedStats[algorithm]["angle"][angleNames[angle]]["tpr"] = 0
        verifiedStats[algorithm]["angle"][angleNames[angle]]["fnr"] = 0
    for subject in verifiedDict[algorithm].keys():
        for angle in verifiedDict[algorithm][subject].keys():
            for comparison in verifiedDict[algorithm][subject][angle]:
                if comparison:
                    verifiedStats[algorithm]["angle"][angle]["tpr"] = verifiedStats[algorithm]["angle"][angle]["tpr"] + 1
                    verifiedStats[algorithm]["tpr"] = verifiedStats[algorithm]["tpr"] + 1
                else:
                    verifiedStats[algorithm]["angle"][angle]["fnr"] = verifiedStats[algorithm]["angle"][angle]["fnr"] + 1
                    verifiedStats[algorithm]["fnr"] = verifiedStats[algorithm]["fnr"] + 1

Overall FPR and TNR by algorithm:

In [6]:
print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + str(verifiedStats[algorithm]["fnr"]))
    print("TP:  " + str(verifiedStats[algorithm]["tpr"]))  
    print("FNMR:  " + str(round(float(verifiedStats[algorithm]["fnr"])/float(verifiedStats[algorithm]["fnr"] + verifiedStats[algorithm]["tpr"]) * 100, 3)) + "%")  


FNMR Results:

Facenet
FN:  72590
TP:  78250
FNMR:  48.124%

VGG-Face
FN:  38820
TP:  112020
FNMR:  25.736%

OpenFace
FN:  59486
TP:  91354
FNMR:  39.436%

ArcFace
FN:  87767
TP:  63073
FNMR:  58.185%

FaceAPI
FN:  33301
TP:  117539
FNMR:  22.077%


In [13]:
for algorithm in verifiedStats.keys():
    for angle in verifiedStats[algorithm]["angle"].keys():
        verifiedStats[algorithm]["angle"][angle]["fnmr"] = round((float(verifiedStats[algorithm]["angle"][angle]["fnr"])/(float(verifiedStats[algorithm]["angle"][angle]["fnr"] + verifiedStats[algorithm]["angle"][angle]["tpr"]))) * 100, 3)


print("FNMR Results:")
for algorithm in verifiedStats.keys():
    print()
    print(algorithm)
    print("FN:  " + "\t-90: " + str(verifiedStats[algorithm]["angle"]["-90"]["fnr"]) + "\t-75: " + str(verifiedStats[algorithm]["angle"]["-75"]["fnr"]) + "\t-60: " + str(verifiedStats[algorithm]["angle"]["-60"]["fnr"]) + "\t-45: " + str(verifiedStats[algorithm]["angle"]["-45"]["fnr"]) + "\t-30: " + str(verifiedStats[algorithm]["angle"]["-30"]["fnr"]) + "\t-15: " + str(verifiedStats[algorithm]["angle"]["-15"]["fnr"]) + "\t15: " + str(verifiedStats[algorithm]["angle"]["15"]["fnr"]) + "\t30: " + str(verifiedStats[algorithm]["angle"]["30"]["fnr"]) + "\t45: " + str(verifiedStats[algorithm]["angle"]["45"]["fnr"]) + "\t60: " + str(verifiedStats[algorithm]["angle"]["60"]["fnr"]) + "\t75: " + str(verifiedStats[algorithm]["angle"]["75"]["fnr"]) + "\t90: " + str(verifiedStats[algorithm]["angle"]["90"]["fnr"]))
    print("TP:  " + "\t-90: " + str(verifiedStats[algorithm]["angle"]["-90"]["tpr"]) + "\t-75: " + str(verifiedStats[algorithm]["angle"]["-75"]["tpr"]) + "\t-60: " + str(verifiedStats[algorithm]["angle"]["-60"]["tpr"]) + "\t-45: " + str(verifiedStats[algorithm]["angle"]["-45"]["tpr"]) + "\t-30: " + str(verifiedStats[algorithm]["angle"]["-30"]["tpr"]) + "\t-15: " + str(verifiedStats[algorithm]["angle"]["-15"]["tpr"]) + "\t15: " + str(verifiedStats[algorithm]["angle"]["15"]["tpr"]) + "\t30: " + str(verifiedStats[algorithm]["angle"]["30"]["tpr"]) + "\t45: " + str(verifiedStats[algorithm]["angle"]["45"]["tpr"]) + "\t60: " + str(verifiedStats[algorithm]["angle"]["60"]["tpr"]) + "\t75: " + str(verifiedStats[algorithm]["angle"]["75"]["tpr"]) + "\t90: " + str(verifiedStats[algorithm]["angle"]["90"]["tpr"]))
    print("FNMR:  " + "\t-90: " + str(verifiedStats[algorithm]["angle"]["-90"]["fnmr"]) + "%\t-75: " + str(verifiedStats[algorithm]["angle"]["-75"]["fnmr"]) + "%\t-60: " + str(verifiedStats[algorithm]["angle"]["-60"]["fnmr"]) + "%\t-45: " + str(verifiedStats[algorithm]["angle"]["-45"]["fnmr"]) + "%\t-30: " + str(verifiedStats[algorithm]["angle"]["-30"]["fnmr"]) + "%\t-15: " + str(verifiedStats[algorithm]["angle"]["-15"]["fnmr"]) + "%\t15: " + str(verifiedStats[algorithm]["angle"]["15"]["fnmr"]) + "%\t30: " + str(verifiedStats[algorithm]["angle"]["30"]["fnmr"]) + "%\t45: " + str(verifiedStats[algorithm]["angle"]["45"]["fnmr"]) + "%\t60: " + str(verifiedStats[algorithm]["angle"]["60"]["fnmr"]) + "%\t75: " + str(verifiedStats[algorithm]["angle"]["75"]["fnmr"]) + "%\t90: " + str(verifiedStats[algorithm]["angle"]["90"]["fnmr"]) + "%")

FNMR Results:

Facenet
FN:  	-90: 9130	-75: 9099	-60: 8697	-45: 5606	-30: 2688	-15: 1128	15: 903	30: 2401	45: 5893	60: 8679	75: 9187	90: 9179
TP:  	-90: 3440	-75: 3471	-60: 3873	-45: 6964	-30: 9882	-15: 11442	15: 11667	30: 10169	45: 6677	60: 3891	75: 3383	90: 3391
FNMR:  	-90: 72.633%	-75: 72.387%	-60: 69.189%	-45: 44.598%	-30: 21.384%	-15: 8.974%	15: 7.184%	30: 19.101%	45: 46.881%	60: 69.045%	75: 73.087%	90: 73.023%

VGG-Face
FN:  	-90: 4365	-75: 6204	-60: 5531	-45: 2177	-30: 780	-15: 565	15: 398	30: 624	45: 2792	60: 5152	75: 4607	90: 5625
TP:  	-90: 8205	-75: 6366	-60: 7039	-45: 10393	-30: 11790	-15: 12005	15: 12172	30: 11946	45: 9778	60: 7418	75: 7963	90: 6945
FNMR:  	-90: 34.726%	-75: 49.356%	-60: 44.002%	-45: 17.319%	-30: 6.205%	-15: 4.495%	15: 3.166%	30: 4.964%	45: 22.212%	60: 40.986%	75: 36.651%	90: 44.749%

OpenFace
FN:  	-90: 9020	-75: 6331	-60: 7214	-45: 4531	-30: 1327	-15: 631	15: 533	30: 1243	45: 5338	60: 6228	75: 7960	90: 9130
TP:  	-90: 3550	-75: 6239	-60: 5356	-45: 8039	